In [2]:
import numpy as np
from requests import get
from bs4 import BeautifulSoup
from time import sleep,time
from random import randint
from warnings import warn
from IPython.core.display import clear_output

# Set the starting and ending years
start_year = 2012
end_year = 2022

# Set the starting and ending pages
start_page = 1
end_page = 501

# Set the increment for the pages
page_increment = 50

headers = {"Accept-Language": "en-US, en;q=0.5"}
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
genres= []
runtime=[]
# Preparing the monitoring of the loop
start_time = time()
requests = 0

# Loop through the years
for year in range(start_year, end_year+1):
    # Format the year for the URL
    year_url = str(year) + '-01-01,' + str(year) + '-12-31'

    # Loop through the pages
    for page in range(start_page, end_page+1, page_increment):
        # Make the URL
        url = 'https://www.imdb.com/search/title/?release_date=' + year_url + '&sort=num_votes,desc&start=' + str(page)
        # Make a request to the URL
        
        response = get(url, headers = headers)
        # Process the response (e.g., extract data, save to a file, etc.)
        

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print(url)
        print(response)
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 200:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)
                
                # Scrape the runtime
                run = container.find('span', {'class': 'runtime'}).text
                run=run.strip().replace(' min','')
                runtime.append(int(run))
                
                # Scrape the genre
                genre = container.find('span', {'class': 'genre'}).text
                genre = genre.strip().replace('\n', '')
                genres.append(genre)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

https://www.imdb.com/search/title/?release_date=2022-01-01,2022-12-31&sort=num_votes,desc&start=501
<Response [200]>
Request:121; Frequency: 0.07473306706062141 requests/s


In [5]:
print(len(names), len(years),len(imdb_ratings), len(metascores), len(votes), len(genres), len(runtime))

2990 2990 2990 2990 2990 2990 2990


In [6]:
import pandas as pd
data = {
    'name': names,
    'year': years,
    'imdb_rating': imdb_ratings,
    'metascore': metascores,
    'votes': votes,
    'genre' : genres,
    'runtime' : runtime
}
movie=pd.DataFrame(data)

In [7]:
movie.head()

,name,year,imdb_rating,metascore,votes,genre,runtime
0,The Dark Knight Rises,(2012),8.4,78,1707459,"Action, Drama",164
1,Django Unchained,(2012),8.4,81,1557229,"Drama, Western",165
2,The Avengers,(2012),8.0,69,1393706,"Action, Adventure, Sci-Fi",143
3,The Hunger Games,(2012),7.2,68,922420,"Action, Adventure, Sci-Fi",142
4,The Hobbit: An Unexpected Journey,(2012),7.8,58,831915,"Adventure, Fantasy",169


In [8]:
movie.shape

(2990, 7)

In [15]:
movie.isna().sum()

name           0
year           0
imdb_rating    0
metascore      0
votes          0
genre          0
runtime        0
dtype: int64

In [17]:
movie.to_csv('imdb_2012_to_2014.csv', index=False)